<a href="https://colab.research.google.com/github/Ayush-Sharma410/HackToberFest2023/blob/maternal_mortality/Projects/Python/Maternal%20Mortality/Maternal_Mortality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ydata-profiling
!pip install pycaret &> /dev/null
print ("Pycaret installed sucessfully!!")

In [ ]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

In [ ]:
df = pd.read_csv("/content/Maternal Health Risk Data Set.csv")

In [ ]:
scale_mapper = {"high risk":2, "mid risk":1, "low risk":0}
df['RiskLevel']=df['RiskLevel'].replace(scale_mapper)

# Comparing different classification models using Pycaret

In [ ]:
from pycaret.classification import *
setup(data = df, target='RiskLevel')
cm = compare_models()

,Description,Value
0,Session id,6723
1,Target,RiskLevel
2,Target type,Multiclass
3,Original data shape,"(1014, 7)"
4,Transformed data shape,"(1014, 7)"
5,Transformed train set shape,"(709, 7)"
6,Transformed test set shape,"(305, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8097,0.9236,0.8097,0.8124,0.8099,0.7103,0.7113,0.2060
lightgbm,Light Gradient Boosting Machine,0.8054,0.9228,0.8054,0.8079,0.8049,0.7042,0.7059,1.0090
rf,Random Forest Classifier,0.8026,0.9243,0.8026,0.8074,0.8023,0.7002,0.7027,0.3120
xgboost,Extreme Gradient Boosting,0.7955,0.9258,0.7955,0.8010,0.7960,0.6894,0.6916,0.1490
dt,Decision Tree Classifier,0.7814,0.8734,0.7814,0.7895,0.7819,0.6682,0.6712,0.0300
gbc,Gradient Boosting Classifier,0.7758,0.9081,0.7758,0.7790,0.7743,0.6581,0.6607,0.4250
knn,K Neighbors Classifier,0.6685,0.8340,0.6685,0.6849,0.6683,0.4899,0.4954,0.0490
ridge,Ridge Classifier,0.6347,0.0000,0.6347,0.6322,0.6097,0.4329,0.4533,0.0260
qda,Quadratic Discriminant Analysis,0.6347,0.7925,0.6347,0.6470,0.6121,0.4330,0.4560,0.0600
lr,Logistic Regression,0.6291,0.7881,0.6291,0.6295,0.6193,0.4291,0.4369,0.9960


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

# Separating the dependent variable

In [ ]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

# Splitting into train and test dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

# Using Extra Trees Classifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=None, oob_score=False,
                     random_state=0, verbose=0, warm_start=False)

# Predicting the values on test set

In [ ]:
y_pred = clf.predict(X_test)

# Finding the accuracy

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.8423645320197044

In [ ]:
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

In [ ]:
comparison_df

,Actual,Predicted
0,2,2
1,1,0
2,1,0
3,0,0
4,0,0
...,...,...
198,0,0
199,2,2
200,1,1
201,0,0


# Tuning the Hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

scoring = 'accuracy'

grid_search = GridSearchCV(ExtraTreesClassifier(random_state=0), param_grid, scoring=scoring, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Best Hyperparameters:", best_params)
print("Test Accuracy with Best Model:", accuracy)

Best Hyperparameters: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 100}
Test Accuracy with Best Model: 0.8522167487684729


# Analyzing the relationship between variables

In [ ]:
profile = ProfileReport(df, title="Profiling Report")

In [ ]:
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
profile.to_file("maternal_mortality.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]